In [64]:
import pandas as pd

# Load the data
df = pd.read_csv('2015_Street_Tree_Census_-_Tree_Data_20241027.csv')

# Display initial shape
print("Initial shape:", df.shape)

# Identify columns with empty values
empty_columns_original = df.columns[df.isnull().any()].tolist()
print("Columns with empty values (original):", empty_columns_original)

# Fill specified columns based on the 'cstatus' column
df.loc[df['status'] == 'Dead', ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems']] = 'dead'
df.loc[df['status'] == 'Stump', ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems']] = 'stump'
df.loc[df['status'] == 'dead', ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems']] = 'dead'
df.loc[df['status'] == 'stump', ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems']] = 'stump'


# Identify columns with empty values after filling
empty_columns_after_fill = df.columns[df.isnull().any()].tolist()
print("Columns with empty values (after fill):", empty_columns_after_fill)

# Drop rows with any empty values in specified key columns
df_cleaned = df.dropna(subset=empty_columns_after_fill)

# Display cleaned shape
print("Cleaned shape:", df_cleaned.shape)
# Save the cleaned data back to Excel
#df_cleaned.to_csv('cleaned_data.csv', index=False
#Used code for making numeric the information below
#print('root stone', df_cleaned['root_stone'].unique())
#print('sidewalk:', df_cleaned['sidewalk'].unique())
#print('guards:', df_cleaned['guards'].unique())
print('steward:', df_cleaned['steward'].unique())
#print('health:', df_cleaned['health'].unique())
#print('user_type:', df_cleaned['user_type'].unique())

Initial shape: (683788, 45)
Columns with empty values (original): ['health', 'spc_latin', 'spc_common', 'steward', 'guards', 'sidewalk', 'problems', 'council district', 'census tract', 'bin', 'bbl']
Columns with empty values (after fill): ['health', 'spc_latin', 'spc_common', 'guards', 'sidewalk', 'problems', 'council district', 'census tract', 'bin', 'bbl']
Cleaned shape: (674115, 45)
steward: ['None' '1or2' '3or4' 'stump' 'dead' '4orMore']


In [65]:
#translating strings into values for analysis
#binary analysis for column with yes/no option
binary_analysis = ['root_stone', 'root_grate', 'root_other', 'trunk_wire',
                   'trnk_light', 'trnk_other', 'brch_light', 'brch_shoe', 'brch_other']
for name in binary_analysis:
    df_cleaned.loc[df_cleaned[name] == 'No', [name]] = 0
    df_cleaned.loc[df_cleaned[name] == 'Yes', [name]] = 1

#Option for sidewalk problem
df_cleaned.loc[df_cleaned['sidewalk'].isin(['NoDamage', 'dead', 'stump']), ['sidewalk']] = 0
df_cleaned.loc[df_cleaned['sidewalk'] == 'Damage', ['sidewalk']] = 1

#Guards problem or lack of it
df_cleaned.loc[df_cleaned['guards'].isin(['None','Unsure','dead','stump']), ['guards']] = 0
df_cleaned.loc[df_cleaned['guards'] == 'Helpful', ['guards']] = 1
df_cleaned.loc[df_cleaned['guards'] == 'Harmful', ['guards']] = -1

#Stewardship- show importance to community, not sure if helpful but show how much community relates to trees
#maybe for later analysis
df_cleaned.loc[df_cleaned['steward'].isin(['None','Unsure','dead','stump']), ['steward']] = 0
df_cleaned.loc[df_cleaned['steward'] == '1or2', ['steward']] = 1.5
df_cleaned.loc[df_cleaned['steward'] == '3or4', ['steward']] = 3.5
df_cleaned.loc[df_cleaned['steward'] == '4orMore', ['steward']] = 5

#Column to be used as a prediction
df_cleaned.loc[df_cleaned['health'].isin(['Poor','dead','stump']), ['health']] = 0
df_cleaned.loc[df_cleaned['health'] == 'Fair', ['health']] = 0.5
df_cleaned.loc[df_cleaned['health'] == 'Good', ['health']] = 1

#bias analysis user_type if neeeded
df_cleaned.loc[df_cleaned['user_type'] == 'TreesCount Staff', ['user_type']] = 0
df_cleaned.loc[df_cleaned['user_type'] == 'Volunteer', ['user_type']] = 1
df_cleaned.loc[df_cleaned['user_type'] ==  'NYC Parks Staff', ['user_type']] = -1





In [66]:
# Convert 'census tract' to string and remove '.0' if present
df_cleaned['census tract'] = df_cleaned['census tract'].astype(str).str.replace('.0$', '', regex=True)

# Function to modify the census tract based on its length
def modify_census_tract(census_tract):
    modified_tracts = []
    for tract in census_tract:
        parts = list(tract)
        j = len(parts)
        if j > 4:
            part = ['0'] * (j + 1)
            for l in range(j - 2):
                part[l] = parts[l]
            part[j] = parts[j -1]
            part[j-1] = parts[j - 2]
            part[j - 2] = '.'
            modified_tracts.append(''.join(part))
        else:
            modified_tracts.append(tract)
    return modified_tracts

# Apply the modification function to the 'census tract' column
df_cleaned['census tract'] = modify_census_tract(df_cleaned['census tract'])

# Verify the results
print(df_cleaned['census tract'])

# Output the modified DataFrame
# df_cleaned.to_csv('Trees_Quantified.csv', index=False)


C:\Users\Mattias A S Larsson\AppData\Local\Temp\ipykernel_32500\1743614572.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['census tract'] = df_cleaned['census tract'].astype(str).str.replace('.0$', '', regex=True)


0            739
1            973
2            449
3            449
4            165
           ...  
683783       519
683784       707
683785       201
683786    235.02
683787      1341
Name: census tract, Length: 674115, dtype: object


C:\Users\Mattias A S Larsson\AppData\Local\Temp\ipykernel_32500\1743614572.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cleaned['census tract'] = modify_census_tract(df_cleaned['census tract'])


In [68]:
df_cleaned.to_csv('Trees_Quantified.csv', index=False)